# Example problem: Monte-Carlo estimate of $\pi$

<img src="https://upload.wikimedia.org/wikipedia/commons/8/84/Pi_30K.gif" width="25%" align=left alt="PI monte-carlo estimate"/>

## Problem description

Suppose we want to estimate the number $\pi$ using a [Monte-Carlo method](https://en.wikipedia.org/wiki/Pi#Monte_Carlo_methods), i.e. obtain a numerical estimate based on a random sampling approach, and that we want at least single precision floating point accuracy.

We take advantage of the fact that the area of a quarter circle with unit radius is $\pi/4$ and that hence the probability of a randomly chosen point inside a unit square to lie within that circle is $\pi/4$ as well.

So for N randomly chosen pairs $(x, y)$ with $x\in[0, 1)$ and $y\in[0, 1)$ we count the number $N_{circ}$ of pairs that also satisfy $(x^2 + y^2) < 1$ and estimage $\pi \approx 4 \cdot N_{circ} / N$.


## Monte-Carlo estimate with NumPy on a single CPU

* NumPy is the fundamental package for scientific computing with Python (https://numpy.org/).
* It contains a powerful n-dimensional array object and useful random number capabilities.

In [1]:
import numpy

In [3]:
def calculate_pi(size_in_bytes):
    
    """Calculate pi using a Monte Carlo method."""
    
    rand_array_shape = (int(size_in_bytes / 8 / 2), 2)
    
    # 2D random array with positions (x, y)
    xy = numpy.random.uniform(low=0.0, high=1.0, size=rand_array_shape)
    
    # check if position (x, y) is in unit circle
    xy_inside_circle = (xy ** 2).sum(axis=1) < 1

    # pi is the fraction of points in circle x 4
    pi = 4 * xy_inside_circle.sum() / xy_inside_circle.size

    print(f"\nfrom {xy.nbytes / 1e9} GB randomly chosen positions")
    print(f"   pi estimate: {pi}")
    print(f"   pi error: {abs(pi - numpy.pi)}\n")
    
    return pi

### Let's calculate...
Observe how the pi error decreases with an increasing number of randomly chosen positions!

In [5]:
%time pi = calculate_pi(size_in_bytes=10_000_000) # 10 MB
%time pi = calculate_pi(size_in_bytes=100_000_000) # 100 MB
%time pi = calculate_pi(size_in_bytes=1_000_000_000) # 1 GB


from 0.01 GB randomly chosen positions
   pi estimate: 3.1431488
   pi error: 0.00155614641020696

CPU times: user 29.4 ms, sys: 567 µs, total: 30 ms
Wall time: 29 ms

from 0.1 GB randomly chosen positions
   pi estimate: 3.14012672
   pi error: 0.001465933589793078

CPU times: user 209 ms, sys: 47.6 ms, total: 256 ms
Wall time: 256 ms

from 1.0 GB randomly chosen positions
   pi estimate: 3.141434048
   pi error: 0.00015860558979330364

CPU times: user 2.06 s, sys: 455 ms, total: 2.52 s
Wall time: 2.52 s


### Are we already better than single precision floating point resolution?

In [6]:
numpy.finfo(numpy.float32)

finfo(resolution=1e-06, min=-3.4028235e+38, max=3.4028235e+38, dtype=float32)

## We won't be able to scale the problem to several Gigabytes or Terabytes!

In [7]:
%time pi = calculate_pi(size_in_bytes=10_000_000_000)  # 10 GB


from 10.0 GB randomly chosen positions
   pi estimate: 3.1415679424
   pi error: 2.4711189793080734e-05

CPU times: user 20.7 s, sys: 18.3 s, total: 39.1 s
Wall time: 39.2 s


In [8]:
# %time calculate_pi(size_in_bytes=100_000_000_000) # 100 GB
# %time calculate_pi(size_in_bytes=1_000_000_000_000) # 1 TB
# %time calculate_pi(size_in_bytes=10_000_000_000_000) # 10 TB

### Problems

* slowness of the numpy-only single CPU approach! (we could scale the problem using the [multiprocessing](https://docs.python.org/3.8/library/multiprocessing.html) and/or [threading](https://docs.python.org/3.8/library/threading.html) libraries)
* frontend/login node compute resources are shared and CPU, memory (and IO bandwidth) user demands will collide